In [8]:
from ase.cluster import Octahedron
from npl.descriptors import Topologies


system = Octahedron('Pt', 5,1)

top = Topologies()
top.create(system)

In [275]:
from numba import njit
import numba as nb
import numpy as np
import random

atom_features = np.empty((len(connectivity_matrix),2))

@njit
def compute_local_environments(connectivity_matrix, atom_features):

    for i in range(len(connectivity_matrix)):

        a_bonds = np.int16(np.sum(connectivity_matrix[i]*position_symbol_a))
        b_bonds = np.int16(np.sum(connectivity_matrix[i]*position_symbol_b))

        atom_features[i][0] = a_bonds
        atom_features[i][1] = b_bonds

@njit
def compute_local_environment(connectivity_matrix, atom_features, atom_index):

    a_bonds = np.int16(np.sum(connectivity_matrix[atom_index]*position_symbol_a))
    b_bonds = np.int16(np.sum(connectivity_matrix[atom_index]*position_symbol_b))

    atom_features[atom_index][0] = a_bonds
    atom_features[atom_index][1] = b_bonds
    


def compute_atom_feature(atom_features, atom_index):

    coordination_number_offsets = [int(cn*(cn + 1)/2) for cn in range(13)]
    element_offset = 91
    element = np.int16(position_symbol_a[atom_index])

    cn = int(sum(atom_features[atom_index]))
    feature = int(element_offset*element + coordination_number_offsets[cn] + atom_features[atom_index][0])
    
compute_local_environments(connectivity_matrix, atom_features)


In [276]:
%timeit compute_atom_feature(atom_features,10)

6.91 µs ± 67.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [265]:
%timeit compute_atom_feature(atom_features,10)

372 ns ± 3.61 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [261]:
compute_local_environments(connectivity_matrix, atom_features)


139


In [251]:
[int(cn*(cn + 1)/2) for cn in range(13)][12]

78

In [228]:
a = np.array([1,1,1,0])

np.where(a==1)

(array([0, 1, 2]),)

{'Ni': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
 'Pt': array([11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
        28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44,
        45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61,
        62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78])}

In [126]:
for f in range(10):
    system[f].symbol = 'Ni'

position_symbol_a = np.zeros(len(system))
position_symbol_b = np.zeros(len(system))

for atom in system:
    if atom.symbol == list(system.symbols.indices())[0]:
        position_symbol_a[atom.index] = 1
    else:
        position_symbol_b[atom.index] = 1

In [133]:
np.sum(connectivity_matrix[0]*position_symbol_b)

1.0

In [29]:
connectivity_matrix = np.zeros((len(system), len(system)))
for col in top.neighbor_list:
    for row in top.neighbor_list[col]:
        connectivity_matrix[col][row] = 1

In [32]:
np.sum(connectivity_matrix[1])

6.0

In [235]:
from Core.LocalEnvironmentFeatureClassifier import TopologicalEnvironmentClassifier
from Core.LocalEnvironmentCalculator import NeighborCountingEnvironmentCalculator
from Core import Nanoparticle as NP 

p = NP.Nanoparticle()
p.add_atoms(system)

l = NeighborCountingEnvironmentCalculator(p.get_all_symbols())
t = TopologicalEnvironmentClassifier(l, p.get_all_symbols())

In [243]:
%timeit t.compute_atom_feature(p,1, recompute_local_environment=True)

16.8 µs ± 143 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [249]:
%timeit compute_atom_feature(connectivity_matrix, atom_features, 1)

627 ns ± 2.29 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
